# WIP First integration of the new perception module into the framework

* I have now added new optional assumptions `oSNc`, `oSNv`, and `oPF`, as planned in the 2021-10-07 notes.
    * The `oSN*` assumptions introduce the new observation noise standard deviation parameter $\tau$, and `oSNv` also requires the observer's eye height $H_e$ over the ground, which I plan on keeping fixed at 1.5 m for both agent types. The `oPF` assumption also introduces a parameter $\sigma_{\dot{x}}$ for the standard deviation of the speed process noise in the Kalman filter. It also requires priors for position and speed of the other agent, but I consider this rather a scenario parameter than a model parameter (or maybe it should be considered a bit of both I guess).
    * **TODO before finalising this notebook: Document these assumptions in the SCPaper readme.**
* These new assumptions are all implemented in the new class `sc_scenario_perception.Perception`, which also can be configured both to yield the old ideal perception, without any noise or filtering. This has required a number of changes in `SCAgent`, which on the highest level amount to now using `self.perception.perc_oth_state` rather than `self.other_agent.curr_state` as the input to the various behaviour and value calculations in `SCAgent.do_action_update()`. Also changes in `SCAgent.get_predicted_other_state()` and `SCAgent.get_prob_of_current_state_given_beh()`, which previously relied on letting `self.other_agent` do predictions, but which now implement those predictions locally instead, from `self.perception.perc_oth_state`. 
    * **TODO before finalising this notebook: I have done some testing to verify that previous behaviour is preserved as it should be when these new assumptions are disabled; add such tests in this notebook before finalising it and moving on to next steps. Like the baseline kinematics, the sequence of adding `oBEo` etc, and then also `oVAl` and so on. **
* Overall in this, there remains some awkwardness from my somewhat hacky use of `commotions.KinematicState` in `SCAgent` to also hold `signed_CP_dist`, `cs_entry/exit_time`, and `long_acc` attributes. The hackiness was there already previously, but it now becomes a bit more obvious because I need to convert back and forth a bit between the `signed_CP_dist` representation (which is what `sc_scenario_perception.Perception` operates on, and the `pos` array representation, which is how position was originally supposed to be stored in `commotions.KinematicState`.
    * As one part of this, I moved the `get_signed_dist_to_conflict_pt()` function from being an `SCAgent` method, to being a function in `sc_scenario_helper.py` instead, and also added the converse function `get_pos_from_signed_dist_to_conflict_pt()` there.

In this (intermediate) commit, the `__name__ == "__main__"` code in `sc_scenario.py` runs a scenario and model (`oVAoSNvoPFoEA`) with a parameterisation that (on most probabilistic runs) generates the sort of deceleration->speedup behaviour that I have had a hard time getting from the deterministic models. **TODO before finalising this notebook: Make sure to properly document this too.**